# 5. Model Building

## 5.1 Import libraries

In [1]:
# Import necessary libaries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data Transformation
from scipy.stats import boxcox

# Modelling 
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, \
                            mean_absolute_percentage_error, explained_variance_score

# Customization
pd.set_option('display.max_columns', 50)

## 5.2 Read CSV as DataFrame 

In [2]:
# Read csv file 
score_cleaned = pd.read_csv('data/score_cleaned.csv', index_col=0)

# See the dataframe 
score_cleaned

number_of_siblings direct_admission     CCA learning_style  gender  \
0                       0              Yes  Sports         Visual  Female   
1                       2               No  Sports       Auditory  Female   
2                       0              Yes     NaN         Visual    Male   
3                       0               No  Sports       Auditory    Male   
4                       0               No    Arts         Visual  Female   
...                   ...              ...     ...            ...     ...   
13814                   1               No   Clubs         Visual  Female   
13815                   1              Yes     NaN       Auditory    Male   
13816                   1              Yes  Sports       Auditory  Female   
13817                   1               No   Clubs         Visual    Male   
13818                   2              Yes     NaN         Visual    Male   

      tuition  final_test  n_male  n_female   age  hours_per_week  \
0          No        69.0    14.0       2.0  16.0            10.0   
1          No        47.0     4.0      19.0  16.0             7.0   
2          No        85.0    14.0       2.0  15.0             8.0   
3          No        66.0    24.0       3.0  16.0             7.0   
4          No        57.0     9.0      12.0  15.0            11.0   
...       ...         ...     ...       ...   ...             ...   
13814      No        56.0    12.0      14.0  16.0             9.0   
13815     Yes        85.0    17.0       5.0  16.0             7.0   
13816     Yes        76.0     7.0      10.0  15.0             7.0   
13817     Yes        45.0    18.0      12.0  16.0             3.0   
13818      No        87.0    11.0       7.0  16.0             9.0   

       attendance_rate sleep_time wake_time  mode_of_transport  
0                 91.0      22:00      6:00  private transport  
1                 94.0      22:30      6:30  private transport  
2                 92.0      22:30      6:30  private transport  
3                 95.0      21:30      5:30   public transport  
4                 96.0      22:30      6:30  private transport  
...                ...        ...       ...                ...  
13814             96.0      22:00      6:00  private transport  
13815             91.0      22:30      6:30  private transport  
13816             93.0      23:00      7:00               walk  
13817             94.0      23:00      7:00               walk  
13818             91.0      23:00      7:00               walk  

[13819 rows x 15 columns]

## 5.3 Split the data into training and testing sets

In [3]:
# Split the data into training and testing sets
X = score_cleaned.drop(columns='final_test')
y = score_cleaned.final_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=555)

## 5.4 Create a pipeline for different transformations

Summary of the preprocessing techniques for different features:
1. Leave as-is 
    - score.number_of_siblings
    - score.age

2. Encode using binary encoding (Yes=1, No=0)
    - score.direct_admission
    - score.gender
    - score.tuition

3. One-hot encode to avoid any ordinal assumptions
    - score.CCA
    - score.learning_style
    - score.mode_of_transport

4. Standardize and normalize using StandardScaler/Min-Max scaling
    - score.n_male
    - score.n_female
    - score.hours_per_week
    - score.attendance_rate

5. Convert to minutes from a fixed point (e.g. midnight) & encode using a cyclic transformation (sine and cosine)
    - score.sleep_time
    - score.wake_time

In [4]:
# Define columns for transformations 
binary_columns = ['direct_admission', 'gender', 'tuition']
datetime_columns = ['sleep_time', 'wake_time']
one_hot_columns = ['CCA', 'learning_style', 'mode_of_transport']
scaling_columns = ['n_male', 'n_female', 'hours_per_week', 'attendance_rate']

# Define Binary encoding function
def binary_encode(df, columns):
    df_encoded = df.copy()
    for col in columns:
        df_encoded[col] = df_encoded[col].map({'Yes': True, 'No': False, 
                                               'Male': True, 'Female':False
                                              })
    return df_encoded 

# Define Cyclic encoding function 
def cyclic_encode_time(df, columns):
    df_encoded = df.copy()
    for col in columns:
        # Convert string to datetime 
        df_encoded[col] = pd.to_datetime(df_encoded[col], format='%H:%M').dt.time 
    
        # Convert time to total seconds from midnight 
        df_encoded[col + '_minutes'] = df_encoded[col].apply(lambda x: x.hour*60 + x.minute)

        # Apply cyclic encoding 
        df_encoded[col + '_sin'] = np.sin(2 * np.pi * df_encoded[col + '_minutes'] / 1440) # 1440 minutes in a day
        df_encoded[col + '_cos'] = np.cos(2 * np.pi * df_encoded[col + '_minutes'] / 1440)

        # Drop intermediate columns if needed
        df_encoded.drop(columns=[col, col + '_minutes'], inplace=True)
        
    return df_encoded

In [7]:
# Apply binary encoding 
X_train = binary_encode(X_train, binary_columns)
X_test = binary_encode(X_test, binary_columns) 

# Apply normalization 
#X_train[scaling_columns] = X_train[scaling_columns].apply(lambda x: np.log1p(x))
#X_test[scaling_columns] = X_test[scaling_columns].apply(lambda x: np.log1p(x))

# Apply Scaling 
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop='first'), one_hot_columns),
        ('scaling', StandardScaler(), scaling_columns)
    ],
    remainder='passthrough'
)

# Create a pipeline with the preprocessor 
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Fit and tranform the training data
X_train_processed = pipeline.fit_transform(X_train) 

# Transform the testing data 
X_test_processed = pipeline.transform(X_test) 

# Get feature names after transformation
transformed_columns = preprocessor.get_feature_names_out()

# Convert processed data back to DataFrame with column names
X_train_processed = pd.DataFrame(X_train_processed, columns = transformed_columns)
X_test_processed = pd.DataFrame(X_test_processed, columns = transformed_columns)

# Apply cyclic encoding for time columns 
X_train_processed = cyclic_encode_time(X_train_processed, ['remainder__sleep_time', 'remainder__wake_time'])
X_test_processed = cyclic_encode_time(X_test_processed, ['remainder__sleep_time', 'remainder__wake_time'])

In [15]:
# See the DataFrame after preprocessing
X_train_processed

onehot__CCA_Clubs onehot__CCA_Sports onehot__CCA_nan  \
0                  0.0                0.0             0.0   
1                  0.0                0.0             0.0   
2                  0.0                0.0             1.0   
3                  0.0                0.0             1.0   
4                  0.0                1.0             0.0   
...                ...                ...             ...   
9668               1.0                0.0             0.0   
9669               0.0                1.0             0.0   
9670               0.0                0.0             0.0   
9671               0.0                0.0             0.0   
9672               0.0                0.0             1.0   

     onehot__learning_style_Visual onehot__mode_of_transport_public transport  \
0                              0.0                                        0.0   
1                              0.0                                        0.0   
2                              0.0                                        1.0   
3                              0.0                                        1.0   
4                              0.0                                        0.0   
...                            ...                                        ...   
9668                           0.0                                        0.0   
9669                           1.0                                        0.0   
9670                           1.0                                        1.0   
9671                           0.0                                        0.0   
9672                           0.0                                        1.0   

     onehot__mode_of_transport_walk scaling__n_male scaling__n_female  \
0                               0.0        0.478329         -0.737259   
1                               1.0        0.022045          0.612866   
2                               0.0        0.478329          0.312838   
3                               0.0        0.174139         -0.437232   
4                               1.0       -0.890524          0.462852   
...                             ...             ...               ...   
9668                            0.0        0.326234           0.01281   
9669                            0.0       -1.650997          0.912893   
9670                            0.0       -0.586334          1.212921   
9671                            0.0       -1.650997          1.962991   
9672                            0.0        0.478329         -0.587246   

     scaling__hours_per_week scaling__attendance_rate  \
0                  -0.295825                 0.333822   
1                  -0.295825                  -0.2956   
2                   1.503161                 0.711476   
3                  -0.520698                  -0.2956   
4                   1.278288                 0.082053   
...                      ...                      ...   
9668               -0.520698                -1.176791   
9669                1.952908                 0.711476   
9670               -0.070951                 0.711476   
9671                1.053415                  -0.2956   
9672                1.503161                 0.459707   

     remainder__number_of_siblings remainder__direct_admission  \
0                                0                       False   
1                                1                       False   
2                                1                       False   
3                                1                        True   
4                                2                       False   
...                            ...                         ...   
9668                             1                       False   
9669                             2                       False   
9670                             1                        True   
9671                             1                        True   
9672       

In [16]:
# Function to convert columns dtype to 'float'
def convert_cols_to_float(df, col):
    df[col] = df[col].astype('float')
    
# Function to convert columns' dtype to 'int' 
def convert_cols_to_int(df, col):
    df[col] = df[col].astype('int')

# Define columns to be converted to 'int' dtype
int_columns = [#'onehot__CCA_Arts'
       'onehot__CCA_Clubs', 'onehot__CCA_Sports',
       'onehot__CCA_nan', #'onehot__learning_style_Auditory',
       'onehot__learning_style_Visual',
       #'onehot__mode_of_transport_private transport',
       'onehot__mode_of_transport_public transport',
       'onehot__mode_of_transport_walk',
       'remainder__number_of_siblings',
       'remainder__direct_admission', 'remainder__gender',
       'remainder__tuition', 'remainder__age'] 

# Define columns to be converted to 'float' dtype
float_columns = ['scaling__n_male', 'scaling__n_female', 
                'scaling__hours_per_week', 'scaling__attendance_rate']

# Convert X_train_processed and X_test_processed columns
convert_cols_to_int(X_train_processed, int_columns)
convert_cols_to_int(X_test_processed, int_columns)
convert_cols_to_float(X_train_processed, float_columns)
convert_cols_to_float(X_test_processed, float_columns)

In [17]:
X_train_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9673 entries, 0 to 9672
Data columns (total 19 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   onehot__CCA_Clubs                           9673 non-null   int64  
 1   onehot__CCA_Sports                          9673 non-null   int64  
 2   onehot__CCA_nan                             9673 non-null   int64  
 3   onehot__learning_style_Visual               9673 non-null   int64  
 4   onehot__mode_of_transport_public transport  9673 non-null   int64  
 5   onehot__mode_of_transport_walk              9673 non-null   int64  
 6   scaling__n_male                             9673 non-null   float64
 7   scaling__n_female                           9673 non-null   float64
 8   scaling__hours_per_week                     9673 non-null   float64
 9   scaling__attendance_rate                    9673 non-null   float64
 10  remainder__n

In [18]:
# Define the models 
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'AdaBoost': AdaBoostRegressor()
}

In [19]:
# Function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae = np.round(mean_absolute_error(y_test, y_pred), 2)
    mse = np.round(mean_squared_error(y_test, y_pred), 2)
    rmse = np.round(np.sqrt(mse),2)
    r2 = np.round(r2_score(y_test, y_pred),2)
    
    return {'MAE':mae, 'MSE':mse, 'RMSE':rmse, 'R2':r2}

In [20]:
# Evaluate each model 
results = {}

for name, model in models.items():
    results[name] = evaluate_model(model, X_train_processed, X_test_processed, y_train, y_test)

# Convert results to DataFrame for better visualization 
results_df = pd.DataFrame(results).T

In [13]:
# Print the results DataFrame
results_df

MAE     MSE   RMSE    R2
Linear Regression  7.22   81.50   9.03  0.59
Ridge Regression   7.23   81.58   9.03  0.59
Lasso Regression   8.32  103.14  10.16  0.48
Decision Tree      7.14  100.88  10.04  0.49
Random Forest      5.45   53.87   7.34  0.73
Gradient Boosting  6.02   60.27   7.76  0.69
AdaBoost           7.54   86.22   9.29  0.56

The best model can be selected based on the following criteria:
- Lower MAE, MSE, RMSE values.
- Higher R2 value (closer to 1).

Based on these criteria:
- <b>Random Forest</b> has the lower MAE (5.44), the lowest MSE (53.65), the lowest RMSE (7.32), and the highest R2 value (0.73). Therefore, the Random Forest model is the best performer among the listed models. The second and third best performing models are <b>Gradient Boosting</b> and <b>Linear Regression/Ridge Regression</b>. 

Note: 
- MAE (Mean Absolute Error) means that, on average, the model's prediction are about X.XX points away from the actual scores. If the scores ranges from 0 to 100, an MAE of ~5.45 might be considered reasonably good.
- MSE (Mean Squared Error) and RMSE (Root Mean Squared Error) penalize larger errors than MAE. X.XX or XX.XX for RMSE indicates the standard deviation of the prediction errors. Again, the acceptability depends on the range of the scores.
- R2 (Coefficient of Determination): An R2 value of X.XX indicates that approximately X.XX% of the variance in the final test scores is explained by the model.

Futher improvements would be done and the following will be considered: 
- Fine-tuning of the hyperparameters of the Random Forest, Gradient Boosting, and the Linear/Ridge Regression models. 

In [39]:
from tpot import TPOTRegressor

# Initialize and fit TPOT
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=555)
tpot.fit(X_train_processed, y_train)

# Evaluate the model
print(tpot.score(X_test_processed, y_test))

/home/brannonoh/.local/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ElasticNetCV(input_matrix, l1_ratio=0.5, tol=0.01)
-81.77667024738909


## 6.1 - Ridge Regression 

I choose Ridge Regression over Linear Regression as Linear Regression has no hyperparameters to be tuned. Therefore, the MAE, MSE, RMSE and R2 values above are final and can't be improved.

In [22]:
# Initlialize the Ridge Linear Regression Model
ridge = Ridge()

# Define the parameter distribution 
param_dist = {'alpha': np.linspace(0.0001, 1, 10),
             'solver': ['svd', 'sparse_cg', 'lsqr', 'sag']
             } 

# Define KFold cross-validation 
kf = KFold(n_splits=5, shuffle=True, random_state=555)

# Initialize RandomizedSearchCV 
ridge_cv = RandomizedSearchCV(estimator=ridge,
                              param_distributions=param_dist,
                              n_iter=30,
                              cv=kf,
                              verbose=1,
                              random_state=555,
                              n_jobs=-1
                             )
                              
# Fit the model 
ridge_cv.fit(X_train_processed, y_train) 

# Get the best parameters & score
print(ridge_cv.best_params_, ridge_cv.best_score_)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
{'solver': 'lsqr', 'alpha': 0.7778} 0.5735668915788127


In [23]:
# Train the final model with the best parameters 
ridge_final = ridge_cv.best_estimator_
ridge_final.fit(X_train_processed, y_train)

# Predict on the test set
y_pred = ridge_final.predict(X_test_processed)

# Evaluation metrics
mae = np.round(mean_absolute_error(y_test, y_pred), 2)
mse = np.round(mean_squared_error(y_test, y_pred), 2)
rmse = np.round(np.sqrt(mse),2)
r2 = np.round(r2_score(y_test, y_pred),2)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R² Score: {r2}')

Mean Absolute Error (MAE): 7.23
Mean Squared Error (MSE): 81.58
Root Mean Squared Error (RMSE): 9.03
R² Score: 0.59


The evaluation metrics do not improve at all. This could be due to the fact that Ridge Regression is not sufficiently complex to capture the underlying patterns in the data as it is still linear. Even with GridSearch, the result is still the same. This is because Ridge Regression is trying to fit a straight line across data that might be too complex for this simple model.

## 6.2 - Gradient Boosting

In [24]:
# Instantiate a GradientBoostingRegressor 
gbr = GradientBoostingRegressor() 

# Define the parameter distribution for RandomizedSearchCV
param_dist = {
    'n_estimators': [300],
    'learning_rate': [0.05],
    'max_depth': [6, 7],
    'min_samples_split': [7],
    'min_samples_leaf': [2],
    'subsample': [0.9], 
}

# Define KFold cross-validation 
kf = KFold(n_splits=5, shuffle=True, random_state=555)

# Initialize RandomizedSearchCV
gbr_cv = RandomizedSearchCV(estimator=gbr,
                            param_distributions=param_dist,
                            n_iter=50,
                            cv=kf,
                            verbose=1,
                            random_state=555
                           )

# Fit the model
gbr_cv.fit(X_train_processed, y_train)

# Get the best parameters & score
print(gbr_cv.best_params_, gbr_cv.best_score_)

/home/brannonoh/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 2 is smaller than n_iter=50. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 2 candidates, totalling 10 fits
{'subsample': 0.9, 'n_estimators': 300, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_depth': 6, 'learning_rate': 0.05} 0.728929091173547


<b>Good to be able to explain why i chose the param_dist, the n_splits, the n_iter in RandomizedSearchCV</b>

1st Try:
- {'subsample': 0.8, 'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 4, 'learning_rate': 0.05} 0.7128179915543796

2nd Try:

- {'subsample': 0.9, 'n_estimators': 300, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_depth': 5, 'learning_rate': 0.05} 0.7258767583247083

3rd Try: 

- {'subsample': 0.9, 'n_estimators': 300, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_depth': 6, 'learning_rate': 0.05} 0.7282037988319471

4th Try: 

- {'subsample': 0.9, 'n_estimators': 300, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_depth': 6, 'learning_rate': 0.05} 0.7280014608201386


After the 4th tuning, I got the best hyperparameters. 

In [25]:
# Train the final model with the best parameters 
gbr_final = gbr_cv.best_estimator_
gbr_final.fit(X_train_processed, y_train)

# Predict on the test set
y_pred = gbr_final.predict(X_test_processed)

# Evaluation metrics
mae = np.round(mean_absolute_error(y_test, y_pred), 2)
mse = np.round(mean_squared_error(y_test, y_pred), 2)
rmse = np.round(np.sqrt(mse),2)
r2 = np.round(r2_score(y_test, y_pred),2)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R² Score: {r2}')

Mean Absolute Error (MAE): 5.34
Mean Squared Error (MSE): 50.27
Root Mean Squared Error (RMSE): 7.09
R² Score: 0.75


The Gradient Boosting model shows an accuracy of 75% when predicting the final test score.

## 6.3 - Random Forest

I will do an experiment. Random Forest with randomized search cross-validation techinque and with Bayesian Optimization.

In [26]:
# Random Forest with RandomizedSearchCV

# Define the parameter distribution for RandomizedSearchCV
param_dist = {
    'n_estimators': [700, 800],
    'max_depth': [30],
    'min_samples_split': [10],
    'min_samples_leaf': [1],
    'bootstrap': [True]
}

# Initialize the Random Forest Regressor model
rf = RandomForestRegressor(random_state=555)

# Define KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=555)

# Initialize RandomizedSearchCV
rf_cv = RandomizedSearchCV(estimator=rf,
                           param_distributions=param_dist,
                           n_iter=50,
                           cv=kf,
                           verbose=1,
                           random_state=555,
                           n_jobs=-1
                          )

# Fit the model
rf_cv.fit(X_train_processed, y_train)

# Get the best params
print(rf_cv.best_params_, rf_cv.best_score_)

/home/brannonoh/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 2 is smaller than n_iter=50. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 2 candidates, totalling 10 fits
{'n_estimators': 700, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 30, 'bootstrap': True} 0.7146952742947654


1st try: 
- {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 20, 'bootstrap': True} 0.7121545984239583

2nd try: 
- {'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 30, 'bootstrap': True} 0.7141177157588193

3rd try: 
- {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 30, 'bootstrap': True} 0.7142640005713339

4th try:
- {'n_estimators': 700, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 30, 'bootstrap': True} 0.7142740768618954

In [27]:
# Train the final model with the best parameters 
rf_cv_final = rf_cv.best_estimator_
rf_cv_final.fit(X_train_processed, y_train)

# Predict on the test set
y_pred = rf_cv_final.predict(X_test_processed)

# Evaluation metrics
mae = np.round(mean_absolute_error(y_test, y_pred), 2)
mse = np.round(mean_squared_error(y_test, y_pred), 2)
rmse = np.round(np.sqrt(mse),2)
r2 = np.round(r2_score(y_test, y_pred),2)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R² Score: {r2}')

Mean Absolute Error (MAE): 5.42
Mean Squared Error (MSE): 53.12
Root Mean Squared Error (RMSE): 7.29
R² Score: 0.73
